In [16]:
from statistics import *
import constants as const
import matplotlib.pyplot as plt
import numpy as np

points = const.POINTS
features = const.FEATURES

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

t1 = file_len("centralized.txt")
central = np.loadtxt("centralized.txt").reshape(t1,features + 1)
w_central = central[:,0:features]
epoch_central = central[:,features]
t2 = file_len("periodic.txt")
periodic = np.loadtxt("periodic.txt").reshape(t2,features + 1)
w_periodic = periodic[:,0:features]
epoch_periodic = periodic[:,features]

norm_central = np.apply_along_axis(np.linalg.norm, 1 ,w_central )
norm_periodic = np.apply_along_axis(np.linalg.norm, 1 ,w_periodic )

plt.plot(epoch_central,norm_central)
plt.plot(epoch_periodic,norm_periodic)
plt.xlabel("unit_time")
plt.ylabel("model_norm")
plt.legend("centralized", "periodic")

UnboundLocalError: local variable 'i' referenced before assignment